# Исследование надежности заемщиков


## Откройте таблицу и изучите общую информацию о данных

Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:

In [65]:
# Импортируем библиотеку
import pandas as pd

In [66]:
# Запишем в переменную данные
data = pd.read_csv('data.csv')

Выведем первые 20 строчек датафрейма `data` на экран.

In [67]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`.

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.

In [69]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [70]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [71]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [72]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [73]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [74]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [75]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями для каждого типа занятости `income_type`.

In [76]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.

In [77]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [78]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверьте остальные столбцы.

In [79]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [80]:
data.duplicated().sum()

71

In [81]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [82]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [83]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [84]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию.**

In [85]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [86]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [87]:
data.head()
# Ваш код будет здесь. Вы можете создавать новые ячейки.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [88]:
#создаем пустой датафрейм для заполнения нужными данными
children_corr = pd.DataFrame()
#считаем общее кол-во людей взявших кредит, в разрезе кол-ва детей
children_corr['total_debt'] = data.groupby('children')['debt'].count() 
#считаем кол-во людей имеющих задолженность по кредиту, в разрезе кол-ва детей
children_corr['unpaid_debt'] = data.groupby('children')['debt'].sum() 
#считаем долю имеющих задолженность людей ко всем взявшим кредит, в разрезе кол-ва детей и приводим получившиеся данные к более наглядному виду
children_corr['ratio'] = round((children_corr['unpaid_debt']/children_corr['total_debt'])*100,2).astype(str) + '%' 
#сортируем и выводим таблицу по убыванию доли должников
children_corr.sort_values('ratio', ascending = False)

,total_debt,unpaid_debt,ratio
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.0%


**Вывод:** 

* Мы наблюдаем, что с увеличением количества детей в семье увеличивается процент задолженности по кредиту.


* Можно предположить, что с ростом количества детей в семье растет процент должников,возможно, ввиду сложной материальной ситуации клиент предпочтет задолжать банку, чем не обеспечить своего ребенка всем необходимым. 


* Хотя мы видим,что семьи с 3 детьми выплачивают кредит чаще чем семьи с 1 и 2 ребенками.


* Реже всех имеют задолженность бездетные семьи, а семьи с 5 детьми и вовсе ее не имеют. 


* И в заключении, хотелось бы сказать, что данные не совсем однозначные, чтобы сделать более точные выводы, возможно, нужна большая выборка, чем та, которую мы имеем.

In [89]:
#создаю сводную таблицу аналогичную выше
children_corr_pivot = data.pivot_table(index = 'children', values='debt', aggfunc = ['count','sum', 'mean'])
#переименовываю колонки согласно таблице выше
children_corr_pivot = children_corr_pivot.rename(columns= {'count':'total_debt', 'sum':'unpaid_debt', 'mean':'ratio'})
#привожу данные к более наглядному виду
children_corr_pivot['ratio'] = round(children_corr_pivot['ratio']*100,2).astype(str) + '%'
#сортирую и вывожу результат на экран
children_corr_pivot

,total_debt,unpaid_debt,ratio
,debt,debt,debt
children,,,
0,14091,1063,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.0%


#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [90]:
#создаем пустой датафрейм для заполнения нужными данными
family_corr = pd.DataFrame()
#считаем общее кол-во людей в каждом сеймейном положении, взявших кредит
family_corr['total_family_status'] = data.groupby('family_status')['debt'].count()
#считаем кол-во людей в каждом сеймейном положении, имеющих задолженность по кредиту
family_corr['unpaid_family_status'] = data.groupby('family_status')['debt'].sum()
#считаем долю имеющих задолженность людей ко всем взявшим кредит, в разрезе сеймейного положения и приводим получившиеся данные к более наглядному виду
family_corr['ratio'] = round((family_corr['unpaid_family_status'] / family_corr['total_family_status'])*100,2).astype(str) + '%'
#сортируем и выводим таблицу по убыванию доли должников
family_corr.sort_values('ratio', ascending = False)


,total_family_status,unpaid_family_status,ratio
family_status,,,
Не женат / не замужем,2796,273,9.76%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%


**Вывод:** 

Из полученной выборки, мы наблюдаем, что семейное положение влияет на задолженность по кредиту:

- люди в браке более дисциплинированны в возврате кредита, в то время как люди не женатые/замужние чаще всех допускают просрочки по кредиту,


- при этом люди в разводе или вдовцы/вдовы чаще платят в срок, чем люди в браке

In [91]:
#создаю сводную таблицу аналогичную выше
family_corr_pivot = data.pivot_table(index = 'family_status', values='debt', aggfunc = ['count','sum', 'mean'])
#переименовываю колонки согласно таблице выше
family_corr_pivot = family_corr_pivot.rename(columns= {'count':'total_family_status', 'sum':'unpaid_family_status', 'mean':'ratio'})
#привожу данные к более наглядному виду
family_corr_pivot['ratio'] = round(family_corr_pivot['ratio']*100,2).astype(str) + '%'
#сортирую и вывожу результат на экран
family_corr_pivot

,total_family_status,unpaid_family_status,ratio
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,9.76%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%


#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [92]:
# На основании столбца, который изначально мы создавали с группировкой по доходам я проводил анализ данных: 

 #0–30.000 — 'E';
 #30.001–50.000 — 'D';
 #50.001–200.000 — 'C';
 #200.001–1.000.000 — 'B';
 #1.000.001 и выше — 'A'

#создаем пустой датафрейм для заполнения нужными данными
income_corr = pd.DataFrame()
#считаем общее кол-во людей с доходами по категориям, взявших кредит
income_corr['total_income_cat'] = data.groupby('total_income_category')['debt'].count()
#считаем кол-во людей с доходами по категориям, имеющих задолженность по кредиту
income_corr['unpaid_income_cat'] = data.groupby('total_income_category')['debt'].sum()
#считаем долю имеющих задолженность людей ко всем, взявшим кредит, в разрезе категории дохода и приводим получившиеся данные к более наглядному виду
income_corr['ratio'] = round((income_corr['unpaid_income_cat'] / income_corr['total_income_cat'])*100,2).astype(str) + '%'
#сортируем и выводим таблицу по убыванию доли должников
income_corr.sort_values('ratio', ascending = False)

,total_income_cat,unpaid_income_cat,ratio
total_income_category,,,
E,22,2,9.09%
C,15921,1353,8.5%
A,25,2,8.0%
B,5014,354,7.06%
D,349,21,6.02%


**Вывод:** 

Из полученной выборки зависимость дохода и возврата кредита в срок показал, что риск есть: чем меньше доход, тем чаще всего клиенты могут допускать просрочки, хотя люди с доходом категории D (от 30 тысяч и до 50 тысяч) имеет меньше задолженностей по платежу, чем люди с более высоким заработком.

In [93]:
#создаю сводную таблицу аналогичную выше
income_corr_pivot = data.pivot_table(index = 'total_income_category', values='debt', aggfunc = ['count','sum', 'mean'])
#переименовываю колонки согласно таблице выше
income_corr_pivot = income_corr_pivot.rename(columns= {'count':'total_income_cat', 'sum':'unpaid_income_cat', 'mean':'ratio'})
#привожу данные к более наглядному виду
income_corr_pivot['ratio'] = round(income_corr_pivot['ratio']*100,2).astype(str) + '%'
#сортирую и вывожу результат на экран
income_corr_pivot

,total_income_cat,unpaid_income_cat,ratio
,debt,debt,debt
total_income_category,,,
A,25,2,8.0%
B,5014,354,7.06%
C,15921,1353,8.5%
D,349,21,6.02%
E,22,2,9.09%


#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [94]:
#создаем пустой датафрейм для заполнения нужными данными
purpose_corr = pd.DataFrame()
#считаем общее кол-во людей взявших кредит для определенной цели
purpose_corr['total_purpose_cat'] = data.groupby('purpose_category')['debt'].count()
#считаем кол-во людей взявших кредит для определенной цели, и имеющих при этом задолженность по кредиту
purpose_corr['unpaid_purpose_cat'] = data.groupby('purpose_category')['debt'].sum()
#считаем долю имеющих задолженность людей ко всем, взявшим кредит, в разрезе цели кредита и приводим получившиеся данные к более наглядному виду
purpose_corr['ratio'] = round((purpose_corr['unpaid_purpose_cat'] / purpose_corr['total_purpose_cat'])*100,2).astype(str) + '%'
#сортируем и выводим таблицу по убыванию доли должников
purpose_corr.sort_values('ratio', ascending = False)


,total_purpose_cat,unpaid_purpose_cat,ratio
purpose_category,,,
операции с автомобилем,4279,400,9.35%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%
операции с недвижимостью,10751,780,7.26%


**Вывод:** 

Из полученной выборки можем сделать вывод о том, что при **покупке автомобиля** люди чаще всего сталкиваются с несвоевременным погашением кредита и чаще всего вовремя погашают кредит по **операцициям с недвижимостью**

In [95]:
#создаю сводную таблицу аналогичную выше
purpose_corr_pivot = data.pivot_table(index = 'purpose_category', values='debt', aggfunc = ['count','sum', 'mean'])
#переименовываю колонки согласно таблице выше
purpose_corr_pivot = purpose_corr_pivot.rename(columns= {'count':'total_income_cat', 'sum':'unpaid_income_cat', 'mean':'ratio'})
#привожу данные к более наглядному виду
purpose_corr_pivot['ratio'] = round(purpose_corr_pivot['ratio']*100,2).astype(str) + '%'
#сортирую и вывожу результат на экран
purpose_corr_pivot

,total_income_cat,unpaid_income_cat,ratio
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,9.35%
операции с недвижимостью,10751,780,7.26%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%


#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Пропуски в данных могут появляться по множеству различных причин, можно предположить, что потенциальными вариантами случившегося может быть:

- пользователь не указал о себе сведения или автоматизированная система сбора информации дала сбой
- возможно они были оставлены умышленно, рассчитывая на автозаполнение спецсимволами.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Так как в отличие от среднего значения при аномально высоких или низких значениях не искажается полученный результат в сторону аномалии, а берется такое значение в выборке, что ровно половина элементов больше него, а другая половина — меньше. соотвественно результат вычислений будет более точным

### Шаг 4: общий вывод.

 В ходе анализа взаимосвязи различных факторов на возврат кредита срок мы выяснили, что все 4 фактора той или иной степени оказывают влияние.
 
Перечислим их в порядке наиболее зависмых к наименее зависимым:

 1. Наличие детей - бездетные с наименьшей вероятностью будут иметь задолженность по кредиту, в то время как люди с 1 ребенком и более уже имеют больший риск задолжать.
 
 2. Семейное положение - люди в браке более ответсвенные чем холостые, в то же время люди в разводе и вдовы/вдовцы имеют наименьший риск просрочки платежа.
 
 3. Люди со средним достатком (50 тыс - 200 тыс) совершают меньше просрочек чем с низким (менее 50 тыс) и высоким достатком (более 200 тыс). Первое можно объяснить нехваткой средств, второе - возможным завышением дохода для получения кредита.
 
 4. Операции с недвижимостью несут наименьший риск среди всех целей кредита, в то время как наибольшую долю невозвратов имеют различные операции с автомобилем
